In [1]:
import os, math

In [4]:
model = "swindeeplab_swin_t"
batch_size = 2
val_batch_size = 6
max_epoch = 100

In [6]:
model = "segmenter_vit_deit"
batch_size = 2
val_batch_size = 4
max_epoch = 100

In [7]:
total_itrs = math.ceil(2975.0*max_epoch/batch_size)
val_interval = math.floor(2975.0/batch_size)

cmd = f"python main.py --model {model} --dataset cityscapes --gpu_id 0 --total_itrs {total_itrs} --val_interval {val_interval}  --lr 0.1 --loss_type mixed  --crop_size 768 --crop_val --batch_size {batch_size} --val_batch_size {val_batch_size} --use_amp --output_stride 16 --data_root ./datasets/data/cityscapes"
print(cmd)

python main.py --model segmenter_vit_deit --dataset cityscapes --gpu_id 0 --total_itrs 148750 --val_interval 1487  --lr 0.1 --loss_type mixed  --crop_size 768 --crop_val --batch_size 2 --val_batch_size 4 --use_amp --output_stride 16 --data_root ./datasets/data/cityscapes
